In [1]:
!pip install pypdf langchain langchain-pinecone langchain-openai

  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.5 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.8 MB/s eta 0:00:00
  Using cached distro-1.9.0-py3

In [4]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.9 MB/s eta 0:00:0000:0100:01


In [5]:
import os
import getpass
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_pinecone import PineconeVectorStore

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt = "Enter your OpenAI API Key")
os.environ["PINECONE_API_KEY"] = getpass.getpass(prompt = "Enter your Pinecone API Key")
os.environ["PINECONE_INDEX_NAME"] = getpass.getpass(prompt = "Enter your Pinecone Index Name")

Enter your OpenAI API Key ········
Enter your Pinecone API Key ········
Enter your Pinecone Index Name ········


In [16]:
store = {}

In [17]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [18]:
def get_condensed_question_prompt():
    prompt = """Given the following conversation history and a follow up question, rephrase the follow up question to be a standalone question, in its original language."""
    condensed_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    return condensed_q_prompt

In [44]:
def get_answer_prompt():
    # prompt = """Answer the question based only on the following context and the answer should be professional. 
    # You always answer the with markdown formatting. You will be penalized if you do not answer with markdown when it would be possible. 
    # The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. 
    # You do not support images and never include images. You will be penalized if you render images.
    # If you dont know the answer, Please respond that you dont know the answer in a polite professinal way. 
    # If the context is empty, say your knowledge is limited. Dont cook up the answer on your own. 
    # If the user greets with Hi, Hello, Hey please greet the user professionally and say that you are a chatbot:

    # {context}"""

    prompt = """You are an helpful AI assistant capable of answering the questions based on the following context. 
    You always answer the with markdown formatting. You will be penalized if you do not answer with markdown when it would be possible. 
    The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. 
    You do not support images and never include images. You will be penalized if you render images.
    the answer should be professional. you should answer the question only about the context provided. 
    you should not generate responses apart from the context. 
    If you dont know the answer, Please respond that you dont know the answer in a professinal way. 
    Don't cook up the answer on your own. 
    If the context is empty, inform to the user that your knowledge is limited to awhonn nursing resources. 
    If the user greets with Hi, Hello, and hey, please greet the user professionally and say that you are a chatbot:

    {context}
    """
    answer_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    return answer_prompt

In [45]:
def get_history_aware_retriever():
    pinecone_index_name = os.environ["PINECONE_INDEX_NAME"]
    embeddings=OpenAIEmbeddings()
    pinecone = PineconeVectorStore(embedding=embeddings, index_name=pinecone_index_name)
    retriever = pinecone.as_retriever()
    history_aware_retriever = create_history_aware_retriever(
        ChatOpenAI(temperature=0.2), retriever, get_condensed_question_prompt()
    )
    return history_aware_retriever

In [46]:
def get_rag_chain_with_history():
    question_answer_chain = create_stuff_documents_chain(ChatOpenAI(temperature=0.2), get_answer_prompt())
    rag_chain = create_retrieval_chain(get_history_aware_retriever(), question_answer_chain)
    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    return conversational_rag_chain

In [47]:
rag_chain = get_rag_chain_with_history()

In [48]:
def invoke_rag_chain(input):
    result = rag_chain.invoke(
            {"input":input},
            config={
                "configurable": {"session_id": "local_chat"}
            }   
        )
    return result

In [49]:
response = invoke_rag_chain("What are all the causes of a diaper rash")

In [50]:
response["answer"]

'### Causes of Diaper Rash:\n\n1. **Prolonged Exposure to Wetness:** \n   - The wet environment enclosed by the diaper can lead to skin irritation and rash.\n   \n2. **Friction:**\n   - Friction between the diaper and the skin can contribute to the development of diaper rash.\n   \n3. **Sensitization to Allergens:**\n   - Allergens present in diapers, such as dyes and rubber components, can enhance sensitization and lead to rash.\n   \n4. **Frequent Stooling:**\n   - Infants with frequent loose stools, short gut, or infectious diarrhea are at risk for diaper dermatitis.\n   \n5. **Antibiotic Use:**\n   - Use of antibiotics can disrupt the natural balance of skin flora and contribute to diaper rash.\n   \n6. **Opiate Withdrawal:**\n   - Opiate withdrawal can be a risk factor for diaper dermatitis.\n   \n7. **Abnormal Rectal Sphincter Tone:**\n   - Conditions like extrophy of the bladder or spina bifida can lead to abnormal rectal sphincter tone and increase the risk of diaper rash.\n   

In [51]:
response = invoke_rag_chain("How to prevent from it")

In [52]:
response["answer"]

"### Strategies to Prevent Diaper Rash:\n\n1. **Identify Infants at Risk:**\n   - Recognize infants who are more prone to diaper dermatitis.\n   \n2. **Perform Focused Skin Assessment:**\n   - Use a valid and reliable tool to assess the perianal area for any signs of irritation.\n   \n3. **Encourage Breastfeeding:**\n   - Support and promote breastfeeding, as it can help reduce the risk of diaper rash.\n   \n4. **Consider Superabsorbent Disposable Diapers:**\n   - Use superabsorbent disposable diapers as an alternative to cloth, especially for infants showing signs of diaper dermatitis.\n   \n5. **Frequent Diaper Changes:**\n   - Check for wet or soiled diapers regularly and change them as needed.\n   \n6. **Gentle Cleansing Methods:**\n   - Use appropriate methods to cleanse the diaper area gently based on the infant's gestational age.\n   \n7. **Use Skin Barrier Products:**\n   - Apply petrolatum-based ointments or skin barrier products containing zinc oxide at every diaper change fo

In [54]:
response["context"]

[Document(page_content='DIAPER WIPES\n1. Implement strategies to reduce the risk or severity of \ndiaper dermatitis.\na. Identify infants at risk for diaper dermatitis. \nb. Perform a focused skin assessment of the perianal \narea using a valid and reliable tool. \nc. Encourage and support breastfeeding. \nd. Consider the use of superabsorbent disposable diapers \nas an alternative to cloth, especially in infants with \nsigns of diaper dermatitis. \ne. Check for wet or soiled diapers and change frequently \nas needed.', metadata={'page': 1.0, 'source': '/tmp/pdf_files/NSCQCG4pages.pdf'}),
 Document(page_content='dermatitis treatment, particularly when commercial \nproducts are unavailable, such as in low-resource settings   \nAn Iranian study of 30 newborns who were breastfed and \ndiapered in cloth diapers found that those infants with \ndiaper rash who were treated with topical breast milk three \ntimes daily showed improved anal dermatitis (p =  006) \ncompared with infants treated 

In [55]:
response = invoke_rag_chain("Who is elon musk?")

In [56]:
response["answer"]

'My knowledge is limited to AWHONN nursing resources, and I do not have information about Elon Musk. If you have any questions related to nursing or healthcare, feel free to ask!'